In [4]:
#  Temel importlar
import os
import pandas as pd
import re
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

#  Dizin tanımları
DATA_DIR = "../data/flickr30k"
IMAGE_DIR = os.path.join(DATA_DIR, "images")
CAPTION_FILE = os.path.join(DATA_DIR, "results.csv")  # veya txt/json

#  Caption mapping sözlüğü (image_id → [caption1, caption2...])
mapping = defaultdict(list)

#  Caption verisini oku
df = pd.read_csv(CAPTION_FILE, delimiter='|')
print(df.head())

df[' comment'] = df[' comment'].fillna('')

#  Mapping sözlüğünü doldur
for _, row in df.iterrows():
    img_id = row['image_name'].strip().split('.')[0]
    caption = row[' comment'].strip()
    mapping[img_id].append(f"<start> {caption} <end>")

#  Bir örnek göster
for k in list(mapping.keys())[:1]:
    print(f"{k} => {mapping[k]}")


       image_name  comment_number  \
0  1000092795.jpg               0   
1  1000092795.jpg               1   
2  1000092795.jpg               2   
3  1000092795.jpg               3   
4  1000092795.jpg               4   

                                             comment  
0   Two young guys with shaggy hair look at their...  
1   Two young , White males are outside near many...  
2   Two men in green shirts are standing in a yard .  
3       A man in a blue shirt standing in a garden .  
4            Two friends enjoy time spent together .  
1000092795 => ['<start> Two young guys with shaggy hair look at their hands while hanging out in the yard . <end>', '<start> Two young , White males are outside near many bushes . <end>', '<start> Two men in green shirts are standing in a yard . <end>', '<start> A man in a blue shirt standing in a garden . <end>', '<start> Two friends enjoy time spent together . <end>']


In [5]:
def clean_caption(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # noktalama işaretlerini kaldır
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Mapping sözlüğünü temizle
for k in mapping:
    mapping[k] = [f"<start> {clean_caption(cap)} <end>" for cap in mapping[k]]


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

all_captions = []
for caps in mapping.values():
    all_captions.extend(caps)

tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

# Tokenizer’ı kaydet (ileride tekrar yüklemek için)
import pickle
with open("../utils/flickr30k_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Maksimum uzunluk
max_len = max(len(c.split()) for c in all_captions)
print("Max caption length:", max_len)


Max caption length: 82


In [9]:
import pickle

with save("utils/flickr30k_mapping.pkl", "wb") as f:
    pickle.dump(mapping, f)

print("Mapping başarıyla kaydedildi ✔")


NameError: name 'save' is not defined